<center>
<h1>Working Examples</h1>
<br>
</center> 

**1) Connect to a SQL Database** <br>
We will use an in-memory only SQLite database. An in-memory SQLite database does not have a physical file and exists only in the memory. 

In [3]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

#Setting echo to TRUE enables logging.
#The return value of create_engine() is an instance of Engine, and it represents the interface to the database.

**2) Declare a mapping**

We have 2 steps to perform:
1. Describe the database table
2. Create our own classes that will be mapped to these tables. 

We can combine these 2 steps using a **declarative base class**. We will create the base class, and see how to create a class called **User** that will be mapped to the **Users** table in the database. At the minimum, we have to specify the table name and the primary key column in order to create a class mapped onto some table.

In [4]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

**3) Create a Schema**

We have created a class **User** in which we have defined the information about the table (this is called the **metadata**). We will now create the actual table in the database. 

We can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. Below, we call the MetaData.create_all() method, passing in our Engine as a source of database connectivity.

In [9]:
Base.metadata.create_all(engine)
#the logging data below shows us that the existance of the table users (PRAGMA table_info("users")) was first checked before the 
#create command was issued.

2017-02-17 13:10:23,502 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-02-17 13:10:23,522 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 13:10:23,530 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-02-17 13:10:23,534 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 13:10:23,542 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-02-17 13:10:23,554 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 13:10:23,570 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-02-17 13:10:23,574 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 13:10:23,582 INFO sqlalchemy.engine.base.Engine COMMIT


**4) Instantiating the User class**

Now that we have created the table, let us see how to create objects of the table. We will eventually use these objects in an INSERT statement.

In [12]:
user1 = User(name = "Jane", fullname = "Jane Doe", password = "password@123") #note that we are NOT assigning the primary key

In [13]:
#we can access each attribute of the object as follows:
user1.fullname

'Jane Doe'

In [14]:
user1.name

'Jane'

In [17]:
user1.password

'password@123'

In [18]:
user1.id #note that this attribute is empty

In [19]:
str(user1.id)

'None'

**5) Creating a session** <br>
We will now create a session. This variable will be used to talk to the database.

In [20]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In case you did not set up the Engine as previously mentioned, you can create the session as: <br><br>
<center> Session = sessionmaker() <br><br> </center>
and then configure it with the engine once it becomes available, as below: <br><br>
<center> Session.configure(bind=engine)</center>

The sessionmaker() creates a Session class. Whenever you want to "talk" to the database, you have to **instantiate** the class

In [21]:
session = Session()

<br>**6) Adding and Updating Objects **
<br><B>add()</B> in used to insert values into the table. **query()** is used for selection and updation operations.

In [23]:
session.add(user1)

This will insert the user1 object that we previously created. However, the database will not be updated until a **flush** occurs. Flush occurs when we query the database for "Jane Doe". 

In [26]:
retrievedUser = session.query(User)
retrievedUser.first()

2017-02-17 14:02:42,768 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-17 14:02:42,772 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-02-17 14:02:42,776 INFO sqlalchemy.engine.base.Engine ('Jane', 'Jane Doe', 'password@123')
2017-02-17 14:02:42,784 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
 LIMIT ? OFFSET ?
2017-02-17 14:02:42,788 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(name='Jane', fullname='Jane Doe', password='password@123')>

The logging clearly shows how the INSERT statement is executed **before** the SELECT statement. <br><br><br>
To insert multiple rows into the table, add_all() is used as follows:

In [27]:
session.add_all([
     User(name = 'Abriella', fullname = 'Abriella Peters', password = 'blah'),
     User(name = 'Monica', fullname = 'Monica Geller', password = 'blah'),
     User(name = 'Chandler', fullname = 'Chandler Bing', password = 'blah')])

Say we want to change the password for Jane Doe.

In [28]:
user1.password = "newPassword"

As mentioned before, the data hasn't been committed to the database. If you wish to see the pending transactions:<br> **session.dirty** will tell you all the modified records that are still pending <br>
**session.new** will tell you all the new records that haven't been inserted

In [29]:
session.dirty

IdentitySet([<User(name='Jane', fullname='Jane Doe', password='newPassword')>])

In [31]:
session.new

IdentitySet([<User(name='Chandler', fullname='Chandler Bing', password='blah')>, <User(name='Monica', fullname='Monica Geller', password='xxg527')>, <User(name='Abriella', fullname='Abriella Peters', password='foobar')>])

Let's commit the pending transactions using session.commit

In [34]:
session.commit()

2017-02-17 14:12:19,794 INFO sqlalchemy.engine.base.Engine UPDATE users SET password=? WHERE users.id = ?
2017-02-17 14:12:19,798 INFO sqlalchemy.engine.base.Engine ('newPassword', 1)
2017-02-17 14:12:19,806 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-02-17 14:12:19,810 INFO sqlalchemy.engine.base.Engine ('Abriella', 'Abriella Peters', 'foobar')
2017-02-17 14:12:19,814 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-02-17 14:12:19,822 INFO sqlalchemy.engine.base.Engine ('Monica', 'Monica Geller', 'xxg527')
2017-02-17 14:12:19,826 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-02-17 14:12:19,830 INFO sqlalchemy.engine.base.Engine ('Chandler', 'Chandler Bing', 'blah')
2017-02-17 14:12:19,838 INFO sqlalchemy.engine.base.Engine COMMIT


In [35]:
session.new #empty set

IdentitySet([])

In [37]:
session.dirty #empty set

IdentitySet([])

It is also possible to **rollback** any changes made to the database. <br><br>
Let's change the name of user1 to Emily from Jane. Also, let's create a dummy user just to see how rollback works.

In [39]:
user1.name = "Emily"

In [41]:
dummyUser = User(name = "Joey", fullname = "Joey Tribbiani", password = "blah")
session.add(dummyUser)

In [42]:
session.query(User).all()

2017-02-17 14:18:20,996 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2017-02-17 14:18:21,000 INFO sqlalchemy.engine.base.Engine ('Emily', 1)
2017-02-17 14:18:21,008 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-02-17 14:18:21,012 INFO sqlalchemy.engine.base.Engine ('Joey', 'Joey Tribbiani', 'blah')
2017-02-17 14:18:21,016 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-02-17 14:18:21,016 INFO sqlalchemy.engine.base.Engine ()


[<User(name='Emily', fullname='Jane Doe', password='newPassword')>,
 <User(name='Abriella', fullname='Abriella Peters', password='foobar')>,
 <User(name='Monica', fullname='Monica Geller', password='xxg527')>,
 <User(name='Chandler', fullname='Chandler Bing', password='blah')>,
 <User(name='Joey', fullname='Joey Tribbiani', password='blah')>]

**Note the name change and the record for the dummy user has been inserted.**

In [44]:
session.rollback()

2017-02-17 14:19:31,992 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [45]:
session.query(User).all()

2017-02-17 14:19:41,139 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-17 14:19:41,147 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-02-17 14:19:41,151 INFO sqlalchemy.engine.base.Engine ()


[<User(name='Jane', fullname='Jane Doe', password='newPassword')>,
 <User(name='Abriella', fullname='Abriella Peters', password='foobar')>,
 <User(name='Monica', fullname='Monica Geller', password='xxg527')>,
 <User(name='Chandler', fullname='Chandler Bing', password='blah')>]

**All changes have been rolled back.**